In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
#try:
#    os.chdir("L:\laupodteam\AIOS\Huibert-Jan\Celldynclustering\celldyn_embedder")
#except FileNotFoundError:
#    os.chdir('C:/Users/Huibert-Jan/Documents/Werk/UMCU/celldyn_embedder')    

import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#import pacmap
import umap
#import trimap

#from scipy.stats import chisquare, chi2_contingency, pearsonr
#from scipy.stats import kendalltau,spearmanr, weightedtau, theilslopes, wilcoxon, ttest_rel
#from scipy.spatial import distance
#import dcor

from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler
#from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#from sklearn.decomposition import PCA
#from sklearn.manifold import Isomap, MDS, SpectralEmbedding
from sklearn.manifold import LocallyLinearEmbedding as LLE, TSNE, smacof, trustworthiness
from sklearn.cluster import KMeans, OPTICS, affinity_propagation, AgglomerativeClustering
# add GMM
from sklearn.mixture import GaussianMixture as GMM, BayesianGaussianMixture as BGMM

#from sklearn.metrics import rand_score, adjusted_mutual_info_score, adjusted_rand_score
#from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score
#from sklearn.neighbors import KNeighborsClassifier as KNN
#from sklearn.cluster import KMeans

from hembedder.utils.distance import poincarre_dist, hyperboloid_dist, fractional_distance, Distance
#from hembedder.utils.quality_metrics import CDEmbeddingPerformance
#import numpy.linalg as la
#import torch 

#from numba import njit

#import faiss
import gc

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist

from tqdm import tqdm

from pysr import PySRRegressor
from gplearn.genetic import SymbolicTransformer
from sklego.meta import ZeroInflatedRegressor
from lineartree import LinearTreeRegressor, LinearForestRegressor

from xgboost import XGBRegressor, XGBRFClassifier

from typing import List, Tuple, Iterable, Callable

In [ ]:
celldyn_full = pd.read_feather("L:\laupodteam\AIOS\Bram\data\CellDyn\\artifacts\celldyn_FULL_transformed_miceforest100_df.feather")
meas_columns = [c for c in celldyn_full.columns if ('c_b' in c) | ("COMBO" in c)]
mode_columns = [c for c in celldyn_full.columns if 'c_m' in c]
alrt_columns = [c for c in celldyn_full.columns if 'alrt' in c.lower()]
c_s_columns = [c for c in celldyn_full.columns if 'c_s_' in c.lower()]
celldyn_full.drop(['index'], axis=1, inplace=True)

celldyn_full = celldyn_full.assign(gender=celldyn_full.gender.map({'M':0, 'F':1}))
celldyn_full.dropna(subset=['gender','draw_hour'], axis=0, inplace=True)

## Bootstrapped Clustering

In [ ]:
# a function to run a clustering algorithm in a bootstrapping fashion
# the outcome is a list of clustering models
def clustering_bootstrapping(data: pd.DataFrame, 
                             n_clusters: int=10,
                             n_bootstraps: int=100, 
                             sample_size: int=50000,
                             clusterer: str='kmeans')-> List[Callable]:
    
    model_list = []
    for i in tqdm(range(n_bootstraps)):
        # sample data
        sample = data.sample(sample_size, replace=True)
        # cluster sample
        if clusterer == 'kmeans':
            model = KMeans(n_clusters=n_clusters, random_state=123)
        elif clusterer == 'optics':
            model = OPTICS(min_samples=100, n_jobs=-1)
        elif clusterer == 'hierarchical':
            model = AgglomerativeClustering(n_clusters=n_clusters)
        elif clusterer == 'gmm':
            model = GMM(n_components=n_clusters, random_state=123)
        elif clusterer == 'bgmm':
            model = BGMM(n_components=n_clusters, random_state=123)
        else:
            raise ValueError("Unknown clustering algorithm")
        model.fit(sample)
        model_list.append(model)
    return model_list

In [ ]:
bclusters = clustering_bootstrapping(celldyn_full[meas_columns], n_clusters=10, n_bootstraps=10, sample_size=50000, clusterer='gmm')

In [ ]:
test_set = celldyn_full.sample(10000, replace=True)[meas_columns]

In [ ]:
def assign_clusters(data: pd.DataFrame, 
                    model_list: List[Callable], 
                    clusterer='kmeans') -> pd.DataFrame:
    
    # get cluster labels
    if clusterer in ['kmeans']:
        labels = np.array([model.predict(data) for model in model_list])
        # get cluster centers
        centers = np.array([model.cluster_centers_ for model in model_list])
        # get cluster sizes
        sizes = np.array([np.bincount(label, minlength=centers.shape[1]) for label in labels])
    elif clusterer in ['gmm']:
        labels = np.array([model.predict(data) for model in model_list])
        # get cluster centers
        centers = np.array([model.means_ for model in model_list])
        # get cluster sizes
        sizes = np.array([np.bincount(label, minlength=centers.shape[1]) for label in labels])        
    return labels, centers, sizes 

In [ ]:
labels, centers, sizes = assign_clusters(test_set, bclusters, clusterer='gmm')

In [ ]:
# we want to re-order the cluster labels for each model such that the ordinal similarity between the models is maximized
init_sim = np.sum(np.corrcoef(labels))/labels.shape[0]**2
print(f"Initial similarity: {init_sim}")


#  we want to align the cluster id's based on the cluster centers or the cluster assignmnents
def align_cluster_assignments(labels: np.ndarray, centers: np.ndarray, method: str='centerwise')-> np.ndarray:
    '''
        Centerwise, given, 1..N models, we align the 2,N models based on the cluster centers of model 1.
        I.e. we find the cluster center in model 2 that is closest to the cluster center in model 1 and assign the cluster id's accordingly
    '''
    


    '''
    	We want to re-order the cluster labels for each model such that the ordinal similarity between the models is maximized.
        We express this similarity with MCC (Matthews correlation coefficient)
    '''
    # first order the labels from the first model, use the order index
    # to order the labels of the other models

    # now change the labels of the other models to match the order of the first model


    return True